In [ ]:
!pip install segmentation-models-pytorch

import pandas as pd
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp

from global_config import *
from models.configs_model.best_unet import *
from dataset.ship_dataset import ShipDataset
from models.unet import UNet
from processing.augmentation import train_transform, val_transform


In [ ]:
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True

In [ ]:
df = pd.read_csv(TRAIN_LABEL_PATH)
COEF_DIF = int(len(df)* 0.15)
df.shape, COEF_DIF

In [ ]:

df = pd.read_csv(TRAIN_LABEL_PATH, nrows=COEF_DIF)

num_nan_sample = min([int(len(df) * 0.3)])
nan_samples = df[df["EncodedPixels"].isna()]
drop_subset = nan_samples.sample(n=num_nan_sample, random_state=42)
df = df.drop(drop_subset.index)
df = df.groupby("ImageId")["EncodedPixels"].apply(list).reset_index()

df

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_df

In [ ]:
train_dataset = ShipDataset(train_df, TRAIN_SAMPLE_DIR, train_transform)
valid_dataset = ShipDataset(val_df, TRAIN_SAMPLE_DIR, val_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
from torch.optim import Adam


model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=WEIGHT,
    classes=CLASSES,
    activation=activation,
    encoder_depth=ENCODER_DEPTH,
    # encoder_channel=ENCODER_CHANNELS,
).to(DEVICE)

optimizer = Adam(model.parameters(), lr=LR)

trainer = UNet(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer,
    epochs=EPOCHS,
    device=DEVICE
)
trainer.train(save_path=OUTPUT_UNET)

In [ ]:
trainer.plot_metrics()